# A study of the monthy hall problem

In this notebook I study the monthy hall problem and the importance of the basis assumptions by doing numeric statistical simulations. The monthy hall problem is divided in three stages. In the first stage the contestant chooses (randomly) one of the three doors offered to him. After this, the presenter opens one of the remaining doors that he KNOWS contains a donkey. After this, the contestant is offered to keep his door or change it. The question is if the contestant should keep or change the door in order to maximize his winning chances. Also, we'll check if the assumption of the presenter knowing what is behind the doors really matters.

#### Bayes Theorem

$$P(A\mid B) = \frac{P(B \mid A) P(A)}{P(B)}$$

## Case 1: host knows what's behind the doors

We are interested in the posterior probability $P(A\mid B)\equiv P(car=1\mid opens=2)$, i.e. the probability that the car is in the door 1 given that Monthy opens, for example, door 2. In order compute that we ened to calculate the Likelihoods:


$$\begin{aligned}&P(opens=2\mid car=1)= 1/2\\&P(opens=2\mid car=2)= 0\\&P(opens=2\mid car=3)= 1\\
\end{aligned}$$

We know that Monthy must open a door without the car behind it:

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [10]:
def host(win, pick):
    '''the host must open a door without the car in it'''
    doors={1,2,3}
    if win!= pick:
        doors.remove(win)
        doors.remove(pick)
    else:
        doors.remove(win)
    door=random.sample(doors,k=1)[0]
    return door

In [11]:
def monthy_hall(change_door=True,verbose=False):
    doors=[1,2,3]
    wining_door=np.random.randint(1,4) #
    contestant_door=np.random.randint(1,4) #
    host_door=host(wining_door,contestant_door)
    doors.remove(host_door)#now the contestan can either change or keep the door
    #change_door=False
    if change_door:
        doors.remove(contestant_door)
        contestant_door=doors[0]
    winner=False
    if (contestant_door==wining_door):
        winner=True
        if verbose:
            print('Winner!\n')
    else:
        if verbose:
            print('Loser!\n')
    return winner

In [12]:
i=0
n_tries=100000
wins=0
for i in np.arange(1,n_tries):
    if monthy_hall(change_door=True):
        wins+=1
win_chance=wins/n_tries
print('Win chance= ' ,win_chance)

Win chance=  0.67021


Win chance approaches 2/3 as expected, if the contestant changes the door.

## Case 2: host randomly picks the door to open

In this case we are in the situation where the host randomply opens a door, and the door happens to be one with a donkey behhind it. 

In [13]:
def random_host(win,pick):
    doors=[1,2,3]
    doors.remove(pick)
    host_door=random.sample(doors,k=1)[0]
    return host_door

In [14]:
def monthy_hall_random(change_door=True,verbose=False):
    '''in this implementation instead we assume that the host picks randomly. However we restrict to the cases in which the host opens (randomly) a door with the donkey in it'''
    doors=[1,2,3]
    wining_door=np.random.randint(1,4) #the car is randomply behind one door
    contestant_door=np.random.randint(1,4) #the ocntestant picks a door
    if(verbose):
        print('Door picked by contestant: ', contestant_door,'\nWinning door: ', wining_door,'\n')
    #host_door=host(wining_door,contestant_door)
    host_door=random_host(wining_door,contestant_door)
    #now i've got to split and consider only the case where the host hasn't picked the car door
    game=False 
    doors.remove(host_door)
    if verbose:
        print('Doors remaining: ', doors)
        print('Host opens door ', host_door, '\n')
    if host_door==wining_door:
        if verbose:
            print('Host has picked the winning door! Game invalid.\n')
        game=False
    else:
        game=True
    if change_door:
        doors.remove(contestant_door)
        contestant_door=doors[0]
        if verbose:
            print('Contestant changes door and picks door ', contestant_door,'\n')
    winner=False
    if (contestant_door==wining_door):
        winner=True
        if verbose:
            print('Winner!\n')
    else:
        if verbose:
            print('Loser!\n')
    return game, winner

In [15]:
monthy_hall_random(verbose=True)

Door picked by contestant:  3 
Winning door:  3 

Doors remaining:  [1, 3]
Host opens door  2 

Contestant changes door and picks door  1 

Loser!



(True, False)

In this case the host picks randomly the door to open between the doors not picked by the contestant. The formulation of the problem is therefore different: "The contestant picks one door: the host opens one door randomly and reveals a donkey, then what are the chances that the car is behind either of the remaining doors, the one picked by the contestant or the other?

I therefore have to restrict on the subset of "valid games" i.e. the games in which the assumption that the host doesn't reveal the car holds.

In Bayesian speak, what changes is the likelyhood probability: Monthy now *can* open a door with the car behind it, therefore, assuming we pick door 1 and monthy opens door 2:

$$\begin{aligned}&P(opens=2\mid car=1)= 1/3\\&P(opens=2\mid car=2)= 1/3\\&P(opens=2\mid car=3)= 1/3\\
\end{aligned}$$

However in this case we are interested in the subset of cases in which Monthy does not open a door with the car behind it, despite choosing randomly. So the question is: *given that monthy did not open a door with the car in it, what are the chances that the car is in either of the remaining doors? Does the chance of winning increase by changing door?*

In [16]:
i=0
n_tries=100000

wins=0
games=0
for i in np.arange(1,n_tries):
    game,win=monthy_hall_random(change_door=True)
    if game:
        games+=1
        if win:
            wins+=1
    #if (win and game):
        #wins+=1
print('Total games: ', n_tries, '\nGames where the host does not pick the winning door (valid games): ', games, '\nValid games won: ', wins)
win_chance=wins/games
print('Win chance= ' ,win_chance)

Total games:  100000 
Games where the host does not pick the winning door (valid games):  66882 
Valid games won:  33522
Win chance=  0.5012110881851619


In this case, given that the choice of the host of what door to open is completely random and not conditioned by the position of the car (independent).